In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from keras.models import Model, model_from_json, load_model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import ModelCheckpoint
import numpy as np
import keras
from tqdm import tqdm
import pandas as pd
import pickle
from time import time
#from keras.utils.vis_utils import plot_model

Using TensorFlow backend.


In [3]:
keras.__version__

'2.1.5'

In [4]:
def vectorize(df):
    eng_sentences = list(df['x'])
    fra_sentences = list(df['y'])
    #print(eng_sentences[:5])
    nb_samples = df.shape[0]
    tokenized_eng_sentences = np.zeros(shape = (nb_samples,max_len_eng_sent,len(eng_chars)), dtype='float32')
    tokenized_fra_sentences = np.zeros(shape = (nb_samples,max_len_fra_sent,len(fra_chars)), dtype='float32')
    target_data = np.zeros((nb_samples, max_len_fra_sent, len(fra_chars)),dtype='float32')
    # Vectorize the english and french sentences

    for i in tqdm(range(nb_samples)):
        for k,ch in enumerate(eng_sentences[i]):
            tokenized_eng_sentences[i,k,eng_char_to_index_dict[ch]] = 1

        for k,ch in enumerate(fra_sentences[i]):
            tokenized_fra_sentences[i,k,fra_char_to_index_dict[ch]] = 1

            # decoder_target_data will be ahead by one timestep and will not include the start character.
            if k > 0:
                target_data[i,k-1,fra_char_to_index_dict[ch]] = 1
    return tokenized_eng_sentences, tokenized_fra_sentences, target_data

In [5]:
def tokenize(seq):
    tokenized_eng_sentence = np.zeros(shape = (1,max_len_eng_sent,len(eng_chars)), dtype='float32')
    for k,ch in enumerate(seq):
        tokenized_eng_sentence[0,k,eng_char_to_index_dict[ch]] = 1
        
    return tokenized_eng_sentence


In [6]:
def decode_seq(inp_seq,encoder_model_inf,decoder_model_inf):
    
    # Initial states value is coming from the encoder 
    states_val = encoder_model_inf.predict(inp_seq)
    
    target_seq = np.zeros((1, 1, len(fra_chars)))
    target_seq[0, 0, fra_char_to_index_dict['\t']] = 1
    
    translated_sent = ''
    stop_condition = False
    
    prob = 1.0
    while not stop_condition:
        
        #decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)
        #predict_pr = decoder_model_inf.predict_proba(x=[target_seq] + states_val)
        decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)
        #print(decoder_out)
        max_val_index = np.argmax(decoder_out[0,-1,:])
        max_val = np.max(decoder_out[0,-1,:])
        prob *= max_val
        sampled_fra_char = fra_index_to_char_dict[max_val_index]
        translated_sent += sampled_fra_char
        #print('{} == {}'.format(sampled_fra_char,max_val))
        
        if ( (sampled_fra_char == '\n') or (len(translated_sent) > max_len_fra_sent)) :
            stop_condition = True
        
        target_seq = np.zeros((1, 1, len(fra_chars)))
        target_seq[0, 0, max_val_index] = 1
        
        states_val = [decoder_h, decoder_c]
    
    prob = prob**(1/len(translated_sent))
    return translated_sent, prob

In [7]:
def fitting(step, model, tokenized_eng_sentences, tokenized_fra_sentences, target_data):
    print('fitting on {} words...'.format(tokenized_eng_sentences.shape[0]))
    t = time()
    model.fit(x=[tokenized_eng_sentences,tokenized_fra_sentences], 
              y=target_data,
              batch_size=64,
              epochs=40,
              validation_split=0.2,
             verbose = 0)
    #print(model.layers)
    print('creating new inference model...')
    # Encoder inference model
    encoder_model_inf = Model(encoder_input, encoder_states)
    # Decoder inference model
    decoder_state_input_h = Input(shape=(256,))
    decoder_state_input_c = Input(shape=(256,))
    decoder_input_states = [decoder_state_input_h, decoder_state_input_c]

    decoder_out, decoder_h, decoder_c = decoder_LSTM(decoder_input, 
                                                     initial_state=decoder_input_states)

    decoder_states = [decoder_h , decoder_c]

    decoder_out = decoder_dense(decoder_out)

    decoder_model_inf = Model(inputs=[decoder_input] + decoder_input_states,
                              outputs=[decoder_out] + decoder_states )
    print('saving inference models...')
    encoder_model_inf.save('retraining_models/encoder-{}.h5'.format(step))
    decoder_model_inf.save('retraining_models/decoder-{}.h5'.format(step))
    model.save('retraining_models/model-{}.h5'.format(step))
    print('fitting took {} min'.format(round(((time()-t)/60),4)))
    return(encoder_model_inf,decoder_model_inf)

In [8]:
#на n-ной итерации берем n*10000 примеров из теста, из них добавляем 10000 к трейну
def testing(encoder_model_inf,decoder_model_inf,train,train_rest,count,step):
    t = time()
    length = max((train_rest).shape[0],step*count) #сколько отрезаем для теста или берем весь
    test = train_rest[:length]
    print('testing on {} words...'.format(test.shape[0]))
    predictions = []
    probs = []
    for row in tqdm(range(test.shape[0])):
        x = test.iloc[row,5]
        x_seq = tokenize(x)
        y,prob = decode_seq(x_seq,encoder_model_inf,decoder_model_inf)
        predictions.append(y.strip())
        probs.append(prob)
    test['predictions'] = predictions
    test['probs'] = probs
    test = test.sort_values(by=['probs'])
    name = 'test_after_{}_step.csv'.format(step)
    test[['x','predictions','probs']].to_csv('training_loop/test/' + name, sep='\t', encoding = 'utf-8')
    test = test[:count] # берем худшие count строк
    #выкидываем из train_rest строки, которые уходят в трейн
    valuelist = test['x']
    train_rest = train_rest[~train_rest.x.isin(valuelist)]
    train = test.copy()
    name_train = 'train_rest_after_{}_step.csv'.format(step)
    train_rest[['x']].to_csv('training_loop/test/' + name_train, sep='\t', encoding = 'utf-8')
    train = test.copy()
    print('testing took {} min'.format(round(((time()-t)/60),4)))
    return train,train_rest
    

### один раз вычитываем данные,  создаем encoder decoder, компилируем модель

In [8]:
total_train = pd.read_csv('data/train/train_full.csv', sep='\t', encoding = 'utf-8')
total_train['x'] = total_train.apply(lambda row: row['lemma'] + str(row['formtag']), axis=1)
total_train['y'] ='\t' + total_train["form"] + '\n'

In [10]:
train_filtered = total_train[total_train.apply(lambda x: not x['classtag'].endswith(('-','+','0')), axis=1)]

In [11]:
data = pickle.load( open( "data.p", "rb" ))
eng_chars = data['eng_chars']
fra_chars = data['fra_chars']
max_len_eng_sent = data['max_len_eng_sent']
max_len_fra_sent = data['max_len_fra_sent']
eng_index_to_char_dict = data['eng_index_to_char_dict']
eng_char_to_index_dict = data['eng_char_to_index_dict']
fra_index_to_char_dict = data['fra_index_to_char_dict']
fra_char_to_index_dict = data['fra_char_to_index_dict']

In [12]:
# Encoder model
encoder_input = Input(shape=(None,len(eng_chars)))
encoder_LSTM = LSTM(256,return_state = True)
encoder_outputs, encoder_h, encoder_c = encoder_LSTM (encoder_input)
encoder_states = [encoder_h, encoder_c]
# Decoder model
decoder_input = Input(shape=(None,len(fra_chars)))
decoder_LSTM = LSTM(256,return_sequences=True, return_state = True)
decoder_out, _ , _ = decoder_LSTM(decoder_input, initial_state=encoder_states)
decoder_dense = Dense(len(fra_chars),activation='softmax')
decoder_out = decoder_dense (decoder_out)
model = Model(inputs=[encoder_input, decoder_input],outputs=[decoder_out])
# Compiling
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
count = 10000
train,train_rest = train_filtered[:count],train_filtered[count:]
step = 1
while train_rest.shape[0] > 0:
    print('iteration: {}, rest of train: {}'.format(step, train_rest.shape[0]))
    tokenized_eng_sentences, tokenized_fra_sentences, target_data = vectorize(train)
    encoder_model_inf,decoder_model_inf = fitting(step, model, tokenized_eng_sentences, tokenized_fra_sentences, target_data) 
    train,train_rest = testing(encoder_model_inf,decoder_model_inf,train,train_rest,count,step)
    step += 1

iteration: 1, rest of train: 373532


100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 65603.14it/s]


fitting on 10000 words...
creating new inference model...
saving inference models...
fitting took 11.3065 min
testing on 373532 words...


100%|█████████████████████████████████████████████████████████████████████████| 373532/373532 [58:44<00:00, 105.98it/s]


testing took 58.793 min
iteration: 2, rest of train: 363532


...

testing took 1.9634 min
iteration: 38, rest of train: 3532


100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 77907.17it/s]


fitting on 10000 words...
creating new inference model...
saving inference models...
fitting took 12.1963 min
testing on 3532 words...


100%|█████████████████████████████████████████████████████████████████████████████| 3532/3532 [00:30<00:00, 114.50it/s]


testing took 0.518 min
